In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [12]:
!pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 12.2 MB/s eta 0:00:0000:0100:01


In [2]:
data= pd.read_csv('~/code/dmassonfr/Spotify_hit_predictor/raw_data/final_dataset_3.csv')

/tmp/ipykernel_26820/2764810032.py:1: DtypeWarning: Columns (22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  data= pd.read_csv('~/code/dmassonfr/Spotify_hit_predictor/raw_data/final_dataset_3.csv')


In [3]:
data.head()

,id,name,popularity,duration_ms,explicit,artists,release_date,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,id_artists,followers,genres,artist_popularity,preview_url,cover_album
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,Uli,1922-02-22,0.645,0.44500,0,...,0.744,0.1510,0.127,104.851,45tIt06XoI0Iio4LBEVpls,91.0,NaN,4,NaN,NaN
1,0PH9AACae1f957JAavhOl2,Lazy Boi,0,157333,0,Uli,1922-02-22,0.298,0.46000,1,...,0.856,0.4360,0.402,87.921,45tIt06XoI0Iio4LBEVpls,91.0,NaN,4,NaN,NaN
2,2SiNuAZ6jIU9xhClRKXcST,Sketch,0,87040,0,Uli,1922-02-22,0.634,0.00399,5,...,0.919,0.1050,0.396,79.895,45tIt06XoI0Iio4LBEVpls,91.0,NaN,4,NaN,NaN
3,4vV7uBcF2AnjNTOejBS5oL,L'enfer,0,40000,0,Uli,1922-02-22,0.657,0.32500,10,...,0.856,0.0931,0.105,81.944,45tIt06XoI0Iio4LBEVpls,91.0,NaN,4,NaN,NaN
4,598LlBn6jpEpVbLjmZPsYV,Graphite,0,104400,0,Uli,1922-02-22,0.644,0.68400,7,...,0.802,0.0847,0.138,100.031,45tIt06XoI0Iio4LBEVpls,91.0,NaN,4,NaN,NaN


In [4]:
data['release_year'] = [int(i.split('-')[0]) for i in data['release_date']]

In [5]:
data=data.drop(columns= ['id_artists','id','name','artists','preview_url','cover_album','genres','release_date'])

In [6]:
data.isna().sum()

popularity           0
duration_ms          0
explicit             0
danceability         0
energy               0
key                  0
loudness             0
mode                 0
speechiness          0
acousticness         0
instrumentalness     0
liveness             0
valence              0
tempo                0
followers            1
artist_popularity    0
release_year         0
dtype: int64

In [7]:
data['followers'] = data['followers'].fillna(0)

In [8]:
y= data['popularity']

In [9]:
X=data.drop(columns= 'popularity')

In [10]:
corr = data.corr()
corr.style.background_gradient(cmap='coolwarm')

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,followers,artist_popularity,release_year
popularity,1.000000,0.028443,0.228952,0.207536,0.301528,0.015706,0.321434,-0.041307,0.003931,-0.380277,-0.229574,-0.043958,-0.001680,0.066489,0.253697,0.525471,0.609825
duration_ms,0.028443,1.000000,-0.020179,-0.124909,0.021004,0.005855,-0.003354,-0.027479,-0.142847,-0.066095,0.067384,0.001178,-0.163686,0.000484,0.029052,0.001952,0.049151
explicit,0.228952,-0.020179,1.000000,0.160505,0.122125,0.010891,0.137307,-0.052012,0.116800,-0.152811,-0.068983,-0.018132,-0.019193,0.006003,0.099597,0.122860,0.221868
danceability,0.207536,-0.124909,0.160505,1.000000,0.247283,0.017833,0.269712,-0.043330,0.192384,-0.239154,-0.231459,-0.107755,0.527828,-0.042363,-0.008211,0.010429,0.242005
energy,0.301528,0.021004,0.122125,0.247283,1.000000,0.035326,0.759741,-0.064690,0.009155,-0.712233,-0.196415,0.132658,0.367858,0.222510,0.060032,0.107409,0.447560
key,0.015706,0.005855,0.010891,0.017833,0.035326,1.000000,0.024430,-0.130758,0.001159,-0.026634,-0.004835,-0.007693,0.019209,0.004558,0.002626,0.001955,0.021702
loudness,0.321434,-0.003354,0.137307,0.269712,0.759741,0.024430,1.000000,-0.041367,-0.090766,-0.519229,-0.336391,0.034082,0.277231,0.182342,0.071273,0.063155,0.432180
mode,-0.041307,-0.027479,-0.052012,-0.043330,-0.064690,-0.130758,-0.041367,1.000000,-0.021424,0.057916,-0.009700,0.010679,0.012857,0.008582,0.004796,0.000078,-0.085150
speechiness,0.003931,-0.142847,0.116800,0.192384,0.009155,0.001159,-0.090766,-0.021424,1.000000,0.060741,-0.101125,0.233273,0.055092,-0.087862,-0.024331,0.125518,-0.014779
acousticness,-0.380277,-0.066095,-0.152811,-0.239154,-0.712233,-0.026634,-0.519229,0.057916,0.060741,1.000000,0.199927,-0.004200,-0.173641,-0.193563,-0.082008,-0.156331,-0.527949


In [13]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

In [14]:
lin_reg = LinearRegression().fit(X, y)
lin_reg.score(X, y)

0.5546482021496981

In [15]:
xgb_reg = XGBRegressor()
parameters = {
    'random_state': [0],
    'reg_alpha': [0],
    'reg_lambda': [1],
    'gamma': [0], #default: 0
    'n_estimators': [1500],
    'learning_rate': [0.05], #default: 0.3
    'colsample_bytree': [0.6], #default: 1
    'subsample': [1], #default: 1
    'max_depth': [10], #default: 6
    'min_child_weight': [4], #default: 1
    'eval_metric': ['rmsle']
    }

### optional to enable CUDA accelerated training
#import torch
#if torch.cuda.device_count() > 0:
#   parameters['tree_method'] = ['gpu_hist']
#   print('CUDA-GPU accelerated training enabled')

In [17]:
clf = GridSearchCV(xgb_reg, parameters, n_jobs=-1)
clf.fit(X, y)
clf.best_params_, clf.best_score_

({'colsample_bytree': 0.6,
  'eval_metric': 'rmsle',
  'gamma': 0,
  'learning_rate': 0.05,
  'max_depth': 10,
  'min_child_weight': 4,
  'n_estimators': 1500,
  'random_state': 0,
  'reg_alpha': 0,
  'reg_lambda': 1,
  'subsample': 1},
 0.4353375524198141)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [19]:
# initialize new XGBRegressor with best parameters from grid search on small dataset
xgb_reg = clf.best_estimator_

# fit on the full dataset
xgb_reg.fit(X_train, y_train, verbose=False)

# Predict against the test set
xgb_pred = xgb_reg.predict(X_test)
    
# Score    
rmse_xgb = mean_squared_error(y_test, xgb_pred, squared=False)
print("Root Mean Squared Error for XGBoost Regressor:", round(rmse_xgb, 3))
r2_xgb = r2_score(y_test, xgb_pred)
print("R2 Score for XGBoost Regressor:", round(r2_xgb, 3))

Root Mean Squared Error for XGBoost Regressor: 9.589
R2 Score for XGBoost Regressor: 0.722
